In [1]:
!/opt/conda/envs/py36/bin/pip install tabulate

In [2]:
import requests
import json

In [3]:
host = 'https://dataverse-dev.ada.edu.au'                  # All clients >4.0 are supported
token = 'enter your token key'  # Generated at /account/apitoken
headers = {'X-Dataverse-key' : token }

In [4]:
# Get the datacerse, datasets and filesi for AES dataverse
url = '{host}/api/search?q=prosecutions&type=dataverse&type=dataset&type=file&per_page=200&sort=name&order=asc'.format(host=host)
r1 = requests.get(url, headers = headers)
data = json.loads(r1.content).get('data')
for item in data['items']:
    print(item['name'], item['url'], item['type'], item.get('identifier', 'None'))
#print(data['items'])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Get the dataset by doi
url = '{host}/api/datasets/:persistentId/?persistentId=doi:10.5072/FK2/JZ2GDI'.format(host=host)
r1 = requests.get(url, headers = headers)
data = json.loads(r1.content).get('data')
#print(json.dumps(data['latestVersion']['files'], indent=4))

In [ ]:
# Get the files for dataset with specified dataset id
# Extract tte dataset id
dsid  = data.get('id')
url = '{host}/api/datasets/{id}/versions/:latest/files'.format(host=host, id=dsid)
r1 = requests.get(url, headers = headers)
data = json.loads(r1.content).get('data')
print(json.dumps(data, indent=4))

In [ ]:
# Get the content of the data file
fileid = data[0].get('dataFile').get('id')
url = '{host}/api/access/datafile/{id}?format=original'.format(host=host, id=fileid)
r1 = requests.get(url, headers = headers)
data = r1.content
#print(data)

In [ ]:
# Save the data
aes_file = open('bigamy_file.csv', mode='wb')
aes_file.write(r1.content)

In [ ]:
# Compute the stats
from collections import defaultdict
from IPython.display import HTML, display
import tabulate
import csv

stats = defaultdict(int)
with open('bigamy_file.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        #print(row['Def_surname'], row['plea'])
        stats[row['plea']] += 1

plea_stats = [[_of, _count] for _of, _count in stats.items()]

_headers = ['Plea to Bigany', 'Total']
display(HTML(tabulate.tabulate(plea_stats, tablefmt='html', headers=_headers)))